In [1]:
import os

In [2]:
%pwd

'd:\\AI Personal Projects\\Machine Learning Projects\\Customer Churn Prediction\\Customer-Churn-Prediction-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\AI Personal Projects\\Machine Learning Projects\\Customer Churn Prediction\\Customer-Churn-Prediction-'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    model: str

In [7]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        model_type = self.params.ModelTrainer.model  
        params = self.params[model_type]             
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name,
            model=model_type
        )

        return model_trainer_config

In [10]:
import pandas as pd 
import os
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from CustomerChurn import logger

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        model_name = self.config.model
        params = read_yaml(PARAMS_FILE_PATH)[model_name]  

        if model_name == 'LogisticRegression':
            model = LogisticRegression(max_iter=params['max_iter'])
        elif model_name == 'RandomForestClassifier':
            model = RandomForestClassifier(
                n_estimators=params['n_estimators'],
                max_depth=params['max_depth'],
                min_samples_split=params['min_samples_split'],
                min_samples_leaf=params['min_samples_leaf'],
                max_features=params['max_features'],
                criterion=params['criterion']
            )
        elif model_name == 'XGBClassifier':
            model = XGBClassifier(
                max_depth=params['max_depth'],
                learning_rate=params['learning_rate'],
                n_estimators=params['n_estimators'],
                subsample=params['subsample'],
                colsample_bytree=params['colsample_bytree']
            )
        else:
            raise ValueError(f"Invalid model name: {model_name}")

        model.fit(X_train, y_train)
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"{model_name} model trained and saved successfully.")

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-07-24 20:26:39,852] 32 CustomerChurnLogger - common - INFO - yaml file: config\config.yaml loaded successfully
[2025-07-24 20:26:39,857] 32 CustomerChurnLogger - common - INFO - yaml file: params.yaml loaded successfully
[2025-07-24 20:26:39,861] 32 CustomerChurnLogger - common - INFO - yaml file: schema.yaml loaded successfully
[2025-07-24 20:26:39,863] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts
[2025-07-24 20:26:39,965] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts/model_trainer
[2025-07-24 20:26:40,328] 32 CustomerChurnLogger - common - INFO - yaml file: params.yaml loaded successfully
[2025-07-24 20:26:41,243] 41 CustomerChurnLogger - 1355951485 - INFO - LogisticRegression model trained and saved successfully.


d:\AI Personal Projects\Machine Learning Projects\Customer Churn Prediction\Customer-Churn-Prediction-\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
